In [1]:
# %pip install tensorflow tensorflow_hub tensorflow_text
# %pip install --upgrade pandas numpy sklearn

In [2]:
import os

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

import pandas as pd
import numpy as np

import pickle


In [3]:
def list_saved_models(root_folder='saved_models'):
    model_paths = []
    for dirpath, dirnames, _ in os.walk(root_folder):
        for dirname in dirnames:
            subfolder_path = os.path.join(dirpath, dirname)
            # Only add the directories which start with 'model'
            if dirname.startswith('model'):
                model_paths.append(subfolder_path)
    return model_paths


model_paths = list_saved_models()


In [4]:
def list_only_not_analyzed_models(model_paths, summary_file='summary.csv'):
    df = pd.read_csv(summary_file, sep='\t')
    analyzed_models = list(df['date'])
    not_analyzed_models = [x for x in model_paths if not any(
        y in x for y in analyzed_models)]
    return not_analyzed_models, df


list_only_not_analyzed_models(model_paths)[0]


['saved_models/small_bert/bert_en_uncased_L-2_H-128_A-2/sigmoid/Adamax/4-epochs/model_2023-04-05_10-46-15']

In [5]:
def analyze_model(model_dir):
    with open(os.path.join(model_dir, 'testing_data.pickle'), 'rb') as f:
        loaded_obj = pickle.load(f)
    _, x_test, _, y_test = loaded_obj.values()
    model = tf.keras.models.load_model(model_dir)
    test_loss, test_acc, test_prec, test_recall = model.evaluate(
        x_test, y_test)
    entries = model_dir.split(sep='/')

    return {'model_name': entries[1],
            'model_parameters': entries[2],
            'activation_func': entries[3],
            'optimization': entries[4],
            'epochs': entries[5].split('-', 1)[0],
            'date': entries[6].split('_', 1)[1],
            'test_loss': test_loss,
            'test_acc': test_acc,
            'test_prec': test_prec,
            'test_recall': test_recall}


In [6]:
def analyze(model_paths, all=False, summary_file='summary.csv'):
    if all:
        return pd.DataFrame(list(map(analyze_model, model_paths)))

    models_to_be_analyzed, analyzed_models = list_only_not_analyzed_models(
        model_paths, summary_file=summary_file)
    newly_analyzed_models = pd.DataFrame(
        list(map(analyze_model, models_to_be_analyzed)))
    analyzed_models.reset_index(drop=True, inplace=True)
    newly_analyzed_models.reset_index(drop=True, inplace=True)
    return pd.concat([analyzed_models, newly_analyzed_models], axis=0)


In [7]:
summary = analyze(model_paths)
summary
# summary = summary.transform(np.sort) # sort by model hierarchy

# summary


35/35 [==============================] - 3s 65ms/step - loss: 0.0369 - binary_accuracy: 0.9901 - precision: 0.9539 - recall: 0.9732


,model_name,model_parameters,activation_func,optimization,epochs,date,test_loss,test_acc,test_prec,test_recall
0,small_bert,bert_en_uncased_L-2_H-128_A-2,sigmoid,Adam,1,2023-04-04_21-29-41,0.027655,0.991928,0.979452,0.959732
1,small_bert,bert_en_uncased_L-2_H-128_A-2,sigmoid,Adam,4,2023-04-04_21-54-19,0.044898,0.991928,1.000000,0.939597
2,small_bert,bert_en_uncased_L-2_H-128_A-2,sigmoid,Adam,5,2023-04-04_21-47-01,0.052441,0.990135,0.992857,0.932886
0,small_bert,bert_en_uncased_L-2_H-128_A-2,sigmoid,Adamax,4,2023-04-05_10-46-15,0.036884,0.990135,0.953947,0.973154


In [8]:
print(list(summary.columns))
summary.sort_values(by=list(summary.columns), ignore_index=True,
                    ascending=True,inplace=True)  # sort by model hierarchy


['model_name', 'model_parameters', 'activation_func', 'optimization', 'epochs', 'date', 'test_loss', 'test_acc', 'test_prec', 'test_recall']


In [9]:
summary.to_csv('summary.csv', sep='\t',index=False)

In [10]:
df = pd.read_csv('summary.csv', sep='\t')
df

,model_name,model_parameters,activation_func,optimization,epochs,date,test_loss,test_acc,test_prec,test_recall
0,small_bert,bert_en_uncased_L-2_H-128_A-2,sigmoid,Adam,1,2023-04-04_21-29-41,0.027655,0.991928,0.979452,0.959732
1,small_bert,bert_en_uncased_L-2_H-128_A-2,sigmoid,Adam,4,2023-04-04_21-54-19,0.044898,0.991928,1.000000,0.939597
2,small_bert,bert_en_uncased_L-2_H-128_A-2,sigmoid,Adam,5,2023-04-04_21-47-01,0.052441,0.990135,0.992857,0.932886
3,small_bert,bert_en_uncased_L-2_H-128_A-2,sigmoid,Adamax,4,2023-04-05_10-46-15,0.036884,0.990135,0.953947,0.973154
